# Visualize Your Inference Output with Gradio Interface

![Gradio example](./images/gradio_example.png)

To quickly evaluate how well the model performs, we provide a visualization tool using [Gradio](https://github.com/gradio-app/gradio), an open-source Python package that allows for the fast deployment of machine learning models with a GUI. Currently, we support the following tasks:

* Segmentation task with `GradioSegmentationStrategy`
* Detection task with `GradioDetectionStrategy`

## Prerequisite

Please follow the instructions in [segmentation example](./seg_example.ipynb).

## Implementation

Let's demonstrate Gradio Interface with [segmentation example](./seg_example.ipynb). To adopt Gradio Interface, we provide `GradioInferAPP` which integrates inference model inherited from `base_gradio_inference_model.py`. By reusing most of the code in [segmentation example](./seg_example.ipynb), we can initiate Gradio Interface seamlessly.

### Setup imports

In [ ]:
import os
from typing import Any, Optional, Sequence, Tuple

import numpy as np
import SimpleITK as sitk
from lungmask import LMInferer

from flavor.serve.apps import GradioInferAPP
from flavor.serve.inference.inference_models import GradioInferenceModel
from flavor.serve.inference.strategies import GradioSegmentationStrategy

### Setup inference model for Gradio Interface

In [ ]:
class SegmentationGradioInferenceModel(GradioInferenceModel):
    def __init__(self):
        super().__init__()

    def define_inference_network(self):
        return LMInferer(modelname="LTRCLobes", fillmodel="R231")

    def set_categories(self):
        categories = [
            {"name": "Background", "display": False},
            {"name": "Left Upper Lobe", "display": True},
            {"name": "Left Lower Lobe", "display": True},
            {"name": "Right Upper Lobe", "display": True},
            {"name": "Right Middle Lobe", "display": True},
            {"name": "Right Lower Lobe", "display": True},
        ]
        return categories

    def set_regressions(self):
        return None

    def data_reader(self, files: Sequence[str], **kwargs) -> Tuple[np.ndarray, None, None]:
        dicom_reader = sitk.ImageFileReader()
        dicom_reader.SetFileName(files[0])
        dicom_reader.ReadImageInformation()
        dicom = sitk.GetArrayFromImage(dicom_reader.Execute()).squeeze()

        data = np.expand_dims(dicom, axis=0)
        return data, None, None

    def preprocess(self, data: np.ndarray) -> np.ndarray:
        return data

    def inference(self, x: np.ndarray) -> np.ndarray:
        return self.network.apply(x)

    def postprocess(self, out: Any, metadata: Optional[Any] = None) -> np.ndarray:
        # (1, h, w) -> (c, h, w)
        out = [
            np.expand_dims((out == i).astype(np.uint8), axis=0)
            for i in range(6)  # or len(self.categories)
        ]
        out = np.concatenate(out, axis=0)
        return out

### Initiate the service

In [ ]:
app = GradioInferAPP(
    infer_function=SegmentationGradioInferenceModel(),
    output_strategy=GradioSegmentationStrategy,
)

app.run(port=int(os.getenv("PORT", 9111)))

## Interact with Gradio interface

You won't need additional JSON file for `GradioInferAPP`.  Go http://localhost:9111/. You can input any images you want and play around with the Gradio GUI running your own inference model. 